In [1]:
# import the necessary packages
from imutils.video import VideoStream
import argparse
import imutils
import time
import cv2 as cv

In [2]:
ap = argparse.ArgumentParser()
ap.add_argument("-v", "--video", type=str,
                help="path to input video file")
ap.add_argument("-t", "--tracker", type=str, default="kcf",
                help="OpenCV object tracker type")
args = vars(ap.parse_args())


usage: ipykernel_launcher.py [-h] [-v VIDEO] [-t TRACKER]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/franel/.local/share/jupyter/runtime/kernel-d86a2b6a-2e2e-4d15-8bd4-f16f48118bef.json


SystemExit: 2

/home/franel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
video_arg = 'example_02.mp4'
tracker_arg = 'kcf'

In [4]:
# initialize a dictionary that maps strings to their corresponding
# OpenCV object tracker implementations
OPENCV_OBJECT_TRACKERS = {
    "csrt": cv.TrackerCSRT_create,
    "kcf": cv.TrackerKCF_create,
    "boosting": cv.TrackerBoosting_create,
    "mil": cv.TrackerMIL_create,
    "tld": cv.TrackerTLD_create,
    "medianflow": cv.TrackerMedianFlow_create,
    "mosse": cv.TrackerMOSSE_create
}
# initialize OpenCV's special multi-object tracker
trackers = cv.MultiTracker_create()

In [5]:
# if a video path was not supplied, grab the reference to the web cam
# if not args.get("video", False):
if not video_arg:
    print("[INFO] starting video stream...")
    vs = VideoStream(src=0).start()
    time.sleep(1.0)
# otherwise, grab a reference to the video file
else:
    # vs = cv2.VideoCapture(args["video"])
    vs = cv.VideoCapture(video_arg)

In [6]:
# loop over frames from the video stream
while True:
    
    # grab the current frame, then handle if we are using a
    # VideoStream or VideoCapture object
    frame = vs.read()
    #frame = frame[1] if args.get("video", False) else frame
    frame = frame[1] if video_arg else frame

    # check to see if we have reached the end of the stream
    if frame is None:
        break

    # resize the frame (so we can process it faster)
    frame = imutils.resize(frame, width=600)
    
    # grab the updated bounding box coordinates (if any) for each
    # object that is being tracked
    (success, boxes) = trackers.update(frame)
    
    # loop over the bounding boxes and draw then on the frame
    for box in boxes:
        (x, y, w, h) = [int(v) for v in box]
        cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
    # show the output frame
    cv.imshow("Frame", frame)
    key = cv.waitKey(1) & 0xFF
    
    # if the 's' key is selected, we are going to "select" a bounding
    # box to track
    if key == ord("s"):
        # select the bounding box of the object we want to track (make
        # sure you press ENTER or SPACE after selecting the ROI)
        box = cv.selectROI("Frame", frame, fromCenter=False,
                           showCrosshair=True)
        
        # create a new object tracker for the bounding box and add it
        # to our multi-object tracker
        #tracker = OPENCV_OBJECT_TRACKERS[args["tracker"]]()
        tracker = OPENCV_OBJECT_TRACKERS[tracker_arg]()
        trackers.add(tracker, frame, box)
        
    # if the `q` key was pressed, break from the loop
    elif key == ord("q"):
        break

# if we are using a webcam, release the pointer
# if not args.get("video", False):
if not video_arg:
    vs.stop()
# otherwise, release the file pointer
else:
    vs.release()

# close all windows
cv.destroyAllWindows()